# 02. Gradient calculation

Most deep learning models are trained using some variation of gradient descent method. The idea behind this method is as follows: 

1. You define loss function - something you want to minimize, e.g. square difference between predicted and actual output. 
2. Then you calculate partial derivatives of this loss function with respect to its parameters. Vector of partial derivatives is called gradient. 
3. Finally, you update the parameters towards minimizing the loss function, where direction and magnitude of the update are proportional to corresponding gradient value. 

For more information on gradient descent see e.g. [this cheatsheet](https://ml-cheatsheet.readthedocs.io/en/latest/gradient_descent.html).

In Avalon you can calculate gradient of any scalar-valued function. For example, consider such a loss function:

In [1]:
function my_loss(y_pred::Vector, y_true::Vector)
    return sum((y_pred .- y_true) .^ 2)
end

y_pred = rand(10)
y_true = rand(10)

my_loss(y_pred, y_true)


1.9112524965207751

Now we can calculate value of the function and gradients w.r.t. to its parameters with a single call to `grad()`. Note that due to function tracing and gradient compilation the first run make take quite long.

In [2]:
using Avalon

val, g = grad(my_loss, y_pred, y_true)

(1.9112524965207751, GradResult(2))

`val` is the same value of the function call that we've seen earlier, and `g` is a `GradResult` object, containing gradients for each parameter:

In [3]:
print(g[1])    # gradient of my_loss w.r.t. 1st parameter, i.e. y_pred
print(g[2])    # gradient of my_loss w.r.t. 2nd parameter, i.e. y_true

[-0.049402581041032345, 0.10532169226954524, 0.35339241886713246, 0.9284107359784595, -0.7659547580536454, -1.5547234645497863, -0.28307026032356264, 0.6702315802893293, -1.7455986003052426, -0.2536506580380271][0.049402581041032345, -0.10532169226954524, -0.35339241886713246, -0.9284107359784595, 0.7659547580536454, 1.5547234645497863, 0.28307026032356264, -0.6702315802893293, 1.7455986003052426, 0.2536506580380271]

In addition to tensors, Avalon can find derivatives w.r.t. scalars (i.e. numbers) and fields of structs. Here's a little bit more realistic code snippet:

In [4]:
mutable struct MyLinear{T}
    W::AbstractArray{T,2}
    b::AbstractArray{T}
end

forward(m::MyLinear, X) = m.W * X

loss(m::MyLinear, X) = sum(forward(m, X))

m = MyLinear(rand(3,4), rand(3))
X = rand(4,5)

val, g = grad(loss, m, X)

print(g[1])
print(g[2])

Dict((:W,) => [1.8337651821026635 1.0853420976456474 2.1428421444148382 2.2434489364594303; 1.8337651821026635 1.0853420976456474 2.1428421444148382 2.2434489364594303; 1.8337651821026635 1.0853420976456474 2.1428421444148382 2.2434489364594303])[0.8768816601703684 0.8768816601703684 0.8768816601703684 0.8768816601703684 0.8768816601703684; 1.7043928946366969 1.7043928946366969 1.7043928946366969 1.7043928946366969 1.7043928946366969; 2.383598200822947 2.383598200822947 2.383598200822947 2.383598200822947 2.383598200822947; 1.6497255912926578 1.6497255912926578 1.6497255912926578 1.6497255912926578 1.6497255912926578]

Note that in case of structs gradient is returned as a dict from field path to its derivative (e.g. `(:W,) => [...]` means that `[...]` is a derivative of loss w.r.t. to field `W` in `MyStruct`). 

Avalon uses computational graph and differntiating capabilities from [Yota.jl](https://github.com/dfdx/Yota.jl), so consult documentation of that package for further information. 